# Regularization 
Agenda today:
- Reviewing overfitting & underfitting, bias variance tradeoff
- Ridge regression 
- Lasso regression 
- AIC and BIC

## Part I. Regularizing a Model
Even though Lasso and Ridge regressions are only used in regression, regularizing a model is a common procedure in the process of building machine learning models. It is an effectuve procedure for tackling the problem of overfitting. Generally speaking, applying regularization technique introduces some **bias** to the model, but reduces the **variance**, and therefore results in better performance in testing data. As you will see later in this module, models built from various classification algorithms often require tuning using regularization in order to overcome overfitting. 

What is regularization in the context of regression? As we recall, as the complexity of model increases, the model overfits and performance on the testing set decreases. Regularization techniques *shrinks* the regression coefficients such that the coefficients are not affecting the outcomes as much as they originally would have. In other words, using regularization applies a *penalty* to the coefficients of your regression model. Let's see how exactly Ridge regression and Lasso regression work to reduce variances in regression models and result in better fit. 

<img src="https://media.giphy.com/media/26ufdipQqU2lhNA4g/giphy.gif" >

## Part II. Ridge Regression (L2 Norm)
Before we dive into regularization, let's (re)visit a concept called **Cost Function**. A cost function is a measure of how good or bad the model is at estimating the relationship of our $X$ and $y$ variables. Usually, it is expressed in the difference between actual values and predicted values. For simple linear regression, the cost function is represented as:
<center> $$ \text{cost_function}= \sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - \sum( bx + b_0))^2$$


For linear regression with multiple predictors, the cost function is expressed as:
$$ \text{cost_function}= \sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - \sum_{j=1}^k(m_jx_{ij} + b))^2$$

Where k stands for number of predictors at jth term.

The ridge regression applies a penalizing parameter $\lambda$ *slope* $^2$, such that a small bias will be introduced to the entire model depending on the value of $\lambda$, which is called a *hyperparameter*. 

$ \text{cost_function_ridge}= \sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - \sum_{j=1}^k(m_jx_{ij} + b))^2 + \lambda \sum_{j=1}^p m_j^2$

The result of applying such a penalizing parameter to the cost function, resulting a different regression model that minimizing the residual sum of square **and** the term $\lambda \sum_{j=1}^p m_j^2$. 

The Ridge regression improves the fit of the original regression line by introducing some bias/changing the slope and intercept of the original line. Recall the way we interpret a regression model Y = mx + b: with every unit increase in x, the outcome y increase by m unit. Therefore, the bigger the coefficient m is, the more the outcome is subjected to changes in predictor x. Ridge regression works by reducing the magnitude of the coefficient m and therefore reducing the effect the predictors have on the outcome. Let's look at a simple example.

The ridge regression penalty term contains all of the coefficients squared from the original regression line except for the intercept term. 

## Part III. Lasso Regression (L1 Norm)
Lasso regression is very similar to Ridge regression except for one difference - the penalty term is not squared but the absolute values of the coefficients muliplied by lambda, expressed by:

$ \text{cost_function_lasso}= \sum_{i=1}^n(y_i - \hat{y})^2 = \sum_{i=1}^n(y_i - \sum_{j=1}^k(m_jx_{ij} + b))^2 + \lambda \sum_{j=1}^p \mid m_j \mid$

The biggest difference in Ridge and Lasso is that Lasso simultaneously performs variable selection: some coefficients are shrunk to 0, rendering them nonexistence in the original regression model. Therefore, Lasso regression performs very well when you have higher dimensional dataset where some predictors are useless; whereas Ridge works best when all the predictors are needed. 

<img src="https://media.giphy.com/media/AWeYSE0qgpk76/giphy.gif" width= "400" />

In [41]:
# implementation 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = sns.load_dataset('mpg')

#data = pd.read_csv("https://raw.githubusercontent.com/learn-co-curriculum/dsc-2-24-09-ridge-and-lasso-regression/master/auto-mpg.csv") 
# data = data.sample(50)
y = data[["mpg"]]
X = data.drop(["mpg", "name", "origin"], axis=1)



In [42]:
data.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64

### Perform a train test split

In [43]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)


In [44]:
X_train.groupby('cylinders')['horsepower'].mean()

cylinders
3    105.000000
4     77.650307
5     90.000000
6     99.984375
8    158.753086
Name: horsepower, dtype: float64

### Handle missing values

In [45]:
cyl_hp = X_train.groupby('cylinders')['horsepower'].mean().round(1).to_dict()
print(cyl_hp)

X_train['horsepower'].fillna(data['cylinders'].map(cyl_hp), inplace=True)


{3: 105.0, 4: 77.7, 5: 90.0, 6: 100.0, 8: 158.8}


/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


### Scale the data by fitting the scaler to the train set and then transforming the train and test set.  

In [46]:
scale = MinMaxScaler()
transformed = scale.fit_transform(X_train)
X_train = pd.DataFrame(transformed, columns = X_train.columns)

In [47]:
transformed = scale.transform(X_test)
X_test = pd.DataFrame(transformed, columns = X_train.columns)

In [48]:
y_train

,mpg
341,23.5
43,13.0
395,32.0
214,13.0
64,15.0
...,...
241,22.0
253,20.5
390,32.0
155,15.0


In [49]:
X_test.isnull().sum()

cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
dtype: int64

In [50]:
X_train.isnull().sum()

cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model_year      0
dtype: int64

### Build a Ridge, Lasso and regular linear regression model. 
***Note how in scikit learn, the regularization parameter is denoted by alpha (and not lambda)***


In [51]:

ridge_01 = Ridge(alpha=0.1)
ridge_01.fit(X_train, y_train)

lasso_01 = Lasso(alpha=0.1)
lasso_01.fit(X_train, y_train)

lin = LinearRegression()
lin.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [52]:
print("Unpenalized Linear Regression Coefficients are:{}".format(lin.coef_))
print(abs(lin.coef_.sum()))
print("Unpenalized Linear Regression Intercept:{}".format(lin.intercept_))

Unpenalized Linear Regression Coefficients are:[[ -1.26235001   2.32556547   1.20626746 -24.66394522   1.96584351
    9.17902821]]
11.24959057720426
Unpenalized Linear Regression Intercept:[26.96714178]


In [53]:
print("Lasso Regression Coefficients are:{}".format(lasso_01.coef_))
print(abs(lasso_01.coef_.sum()))
print("Lasso Linear Regression Intercept:{}".format(lasso_01.intercept_))

Lasso Regression Coefficients are:[ -2.06402443  -0.          -0.         -19.5421625    0.
   8.1966369 ]
13.40955003333538
Lasso Linear Regression Intercept:[27.92234918]


In [54]:
print("Ridge Regression Coefficients are:{}".format(ridge_01.coef_))
print(abs(ridge_01.coef_.sum()))
print("Ridge Linear Regression Intercept:{}".format(ridge_01.intercept_))

Ridge Regression Coefficients are:[[ -1.29886595   1.47005304   0.50375617 -23.26511847   1.46018539
    9.06062809]]
12.069361722977632
Ridge Linear Regression Intercept:[27.23683459]


### Fit models with a different lambda

In [29]:
ridge_05 = Ridge(alpha=0.5)
ridge_05.fit(X_train, y_train)

lasso_05 = Lasso(alpha=0.5)
lasso_05.fit(X_train, y_train)

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [30]:
print("Lasso Regression Coefficients are:{}".format(lasso_05.coef_))
print(lasso_05.coef_.sum())
print("Lasso Linear Regression Intercept:{}".format(lasso_05.intercept_))


Lasso Regression Coefficients are:[-7.935531   -0.         -0.         -6.52878793  0.          4.66684109]
-9.79747784309771
Lasso Linear Regression Intercept:[27.60721812]


In [31]:
print("Ridge Regression Coefficients are:{}".format(ridge_05.coef_))
print(ridge_05.coef_.sum())

print("Ridge Linear Regression Intercept:{}".format(ridge_05.intercept_))

Ridge Regression Coefficients are:[[ -1.60597959  -0.67851021  -1.23543714 -19.41288527   0.26072953
    8.71509092]]
-13.956991768542922
Ridge Linear Regression Intercept:[27.8744965]


In [32]:
ridge_1 = Ridge(alpha=1)
ridge_1.fit(X_train, y_train)

lasso_1 = Lasso(alpha=1)
lasso_1.fit(X_train, y_train)

Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

In [33]:
print("Lasso Regression Coefficients are:{}".format(lasso_1.coef_))
print(lasso_1.coef_.sum())
print("Lasso Linear Regression Intercept:{}".format(lasso_1.intercept_))


Lasso Regression Coefficients are:[-9.0763542  -0.         -0.         -0.          0.          0.40628264]
-8.670071561381564
Lasso Linear Regression Intercept:[27.80325472]


In [34]:
print("Ridge Regression Coefficients are:{}".format(ridge_1.coef_))
print(ridge_1.coef_.sum())
print("Ridge Linear Regression Intercept:{}".format(ridge_1.intercept_))

Ridge Regression Coefficients are:[[ -2.06268246  -2.03325591  -2.27266513 -16.56743472  -0.3738645
    8.42642636]]
-14.883476355648753
Ridge Linear Regression Intercept:[28.20794402]


In [61]:
# create predictions
y_h_ridge_train_01 = ridge_01.predict(X_train)
y_h_ridge_test_01 = ridge_01.predict(X_test)

y_h_lasso_train_01 = np.reshape(lasso_01.predict(X_train),(40,1))
y_h_lasso_test_01 = np.reshape(lasso_01.predict(X_test),(10,1))

y_h_lin_train = lin.predict(X_train)
y_h_lin_test = lin.predict(X_test)

ValueError: cannot reshape array of size 318 into shape (40,1)

In [62]:
print(y_h_ridge_train_01.shape)
print(y_h_ridge_test_01.shape)

(318, 1)
(80, 1)


In [63]:
print(type(y_h_lasso_train_01))
print(type(y_h_ridge_train_01))

NameError: name 'y_h_lasso_train_01' is not defined

#### Examining the Residual for Ridge, Lasso, and Unpenalized Regression coefficients

In [64]:
# examine the residual sum of sq
print('Train Error Ridge Model', np.sum((y_train - y_h_ridge_train_01)**2))
print('Test Error Ridge Model', np.sum((y_test - y_h_ridge_test_01)**2))
print('\n')

print('Train Error Lasso Model', np.sum((y_train - y_h_lasso_train_01)**2))
print('Test Error Lasso Model', np.sum((y_test - y_h_lasso_test_01)**2))
print('\n')

print('Train Error Unpenalized Linear Model', np.sum((y_train - lin.predict(X_train))**2))
print('Test Error Unpenalized Linear Model', np.sum((y_test - lin.predict(X_test))**2))

Train Error Ridge Model mpg    3938.100176
dtype: float64
Test Error Ridge Model mpg    707.169924
dtype: float64




NameError: name 'y_h_lasso_train_01' is not defined

## How does Ridge and Lasso Perform in Higher Dimensional Data?

#### 2 degree polynomials

In [65]:
data.shape

(398, 9)

In [66]:
## try polynomial features on the regression 
from sklearn.preprocessing import PolynomialFeatures

#instantiate this class
poly_2 = PolynomialFeatures(degree=2, interaction_only=False)
#fit and transform the data and create a  new dataframe
df_poly= pd.DataFrame(poly_2.fit_transform(X), columns=poly_2.get_feature_names(X.columns))


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
df_poly.shape

In [ ]:
X_train , X_test, y_train, y_test = train_test_split(df_poly, y, test_size=0.2, random_state=12)


In [ ]:
scale = MinMaxScaler()
transformed = scale.fit_transform(X_train)
X_train = pd.DataFrame(transformed, columns = X_train.columns)

transformed = scale.transform(X_test)
X_test = pd.DataFrame(transformed, columns = X_train.columns)

In [ ]:

# Build a Ridge, Lasso and regular linear regression model. 
# Note how in scikit learn, the regularization parameter is denoted by alpha (and not lambda)
ridge = Ridge(alpha=0.3)
ridge.fit(X_train, y_train)

lasso = Lasso(alpha=0.3)
lasso.fit(X_train, y_train)

lin = LinearRegression()
lin.fit(X_train, y_train)

In [ ]:
print("Unpenalized Linear Regression Coefficients are:{}".format(lin.coef_))
print("Unpenalized Linear Regression Intercept:{}".format(lin.intercept_))

In [ ]:
print("Lasso Regression Coefficients are:{}".format(lasso.coef_))
print("Lasso Linear Regression Intercept:{}".format(lasso.intercept_))

In [ ]:
print("Ridge Regression Coefficients are:{}".format(ridge.coef_))
print("Ridge Linear Regression Intercept:{}".format(ridge.intercept_))

In [ ]:
# create predictions
y_h_ridge_train = ridge.predict(X_train)
y_h_ridge_test = ridge.predict(X_test)

y_h_lasso_train = np.reshape(lasso.predict(X_train),(40,1))
y_h_lasso_test = np.reshape(lasso.predict(X_test),(10,1))

y_h_lin_train = lin.predict(X_train)
y_h_lin_test = lin.predict(X_test)

In [ ]:
# examine the residual sum of sq
print('Train Error Ridge Model', np.sum((y_train - y_h_ridge_train)**2))
print('Test Error Ridge Model', np.sum((y_test - y_h_ridge_test)**2))
print('\n')

print('Train Error Lasso Model', np.sum((y_train - y_h_lasso_train)**2))
print('Test Error Lasso Model', np.sum((y_test - y_h_lasso_test)**2))
print('\n')

print('Train Error Unpenalized Linear Model', np.sum((y_train - lin.predict(X_train))**2))
print('Test Error Unpenalized Linear Model', np.sum((y_test - lin.predict(X_test))**2))

#### Even higher degree polynomials

In [ ]:
poly_5 = PolynomialFeatures(degree=5, interaction_only=False)
#fit and transform the data and create a  new dataframe
df_poly_5= pd.DataFrame(poly_5.fit_transform(X), columns=poly_5.get_feature_names(X.columns))
df_poly_5.shape

In [ ]:
X_train , X_test, y_train, y_test = train_test_split(df_poly_5, y, test_size=0.2, random_state=12)

# Build a Ridge, Lasso and regular linear regression model. 
# Note how in scikit learn, the regularization parameter is denoted by alpha (and not lambda)
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)

lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train)

lin = LinearRegression()
lin.fit(X_train, y_train)

In [ ]:
# create predictions
y_h_ridge_train = ridge.predict(X_train)
y_h_ridge_test = ridge.predict(X_test)

y_h_lasso_train = np.reshape(lasso.predict(X_train),(40,1))
y_h_lasso_test = np.reshape(lasso.predict(X_test),(10,1))

y_h_lin_train = lin.predict(X_train)
y_h_lin_test = lin.predict(X_test)

In [ ]:
# examine the residual sum of sq
print('Train Error Ridge Model', np.sum((y_train - y_h_ridge_train)**2))
print('Test Error Ridge Model', np.sum((y_test - y_h_ridge_test)**2))
print('\n')

print('Train Error Lasso Model', np.sum((y_train - y_h_lasso_train)**2))
print('Test Error Lasso Model', np.sum((y_test - y_h_lasso_test)**2))
print('\n')

print('Train Error Unpenalized Linear Model', np.sum((y_train - lin.predict(X_train))**2))
print('Test Error Unpenalized Linear Model', np.sum((y_test - lin.predict(X_test))**2))

## Calculating AIC and BIC 
AIC and BIC are information criteria for evaluating how good of a model is by giving a measurement of parsimony and goodness of fit. 

- AIC is defined as: $2k - 2log(L)$
- BIC is defined as: $klog(n) - 2log(L)$  

In [ ]:
def aic(y, y_pred, k):
    resid = y - y_pred
    sse = (resid**2).sum()
    AIC = 2*k - 2*np.log(sse)
    
    return AIC

In [ ]:
df_poly_5.shape[1]

In [ ]:
aic(y_test, y_h_lasso_test, df_poly_5.shape[1])

In [ ]:
aic(y_test, y_h_ridge_test, df_poly_5.shape[1])

In [ ]:
aic(y_test, y_h_lin_test, df_poly_5.shape[1])